# Batch Normalization & Dropout 

In this Notebook we will implement Batch Normalization and Dropout and discuss how both these methods help in Neural Network training. 

By completing this exercise you will:
1. Know the implementation details of Batch Norm and Dropout
2. Notice the difference in behaviour during train and test time
3. Use Batch Norm and Dropout in a Fully Connected Layer to see how it affects training

Let us start with Batch Normalization:

# 1. Batch Normalization

## 1.1 What is Batch Normalization
One way to make deep networks easier to train is to use more sophisticated optimization procedures such as SGD+momentum, RMSProp, or Adam. Another strategy is to change the architecture of the network to make it easier to train. One idea along these lines is batch normalization which was proposed by [1].

The idea is relatively straightforward. Machine learning methods tend to work better when their input data consists of uncorrelated features with zero mean and unit variance. When training a neural network, we can preprocess the data before feeding it to the network to explicitly decorrelate its features; this will ensure that the first layer of the network sees data that follows a nice distribution. However even if we preprocess the input data, the activations at deeper layers of the network will likely no longer be decorrelated and will no longer have zero mean or unit variance since they are output from earlier layers in the network. Even worse, during the training process the distribution of features at each layer of the network will shift as the weights of each layer are updated.

The authors of [1] hypothesize that the shifting distribution of features inside deep neural networks may make training deep networks more difficult. To overcome this problem, [1] proposes to insert batch normalization layers into the network. At training time, a batch normalization layer uses a minibatch of data to estimate the mean and standard deviation of each feature. These estimated means and standard deviations are then used to center and normalize the features of the minibatch. A running average of these means and standard deviations is kept during training, and at test time these running averages are used to center and normalize features.

It is possible that this normalization strategy could reduce the representational power of the network, since it may sometimes be optimal for certain layers to have features that are not zero-mean or unit variance. To this end, the batch normalization layer includes learnable shift and scale parameters for each feature dimension.

[1] Sergey Ioffe and Christian Szegedy, "Batch Normalization: Accelerating Deep Network Training by Reducing
Internal Covariate Shift", ICML 2015.

### Before We Start

It is important that we take a look at the Mathematical formula behind Batch Norm. Please make sure to understand the formula below since this will definitely help you with the implementation later.

![](https://drive.google.com/uc?export=view&id=1TJ4_jbyQRPinXmYDqN1mRK9_Vtjt68WI)

### A quick explanation of the formula
Let's summarize the mathematics here: In the left column, we are given the input data, which consists (as always) of $N$ samples of dimension $D$. Furthermore, we need the learnable shift and scale parameters which we call $\beta$ and $\gamma$. The intermediates are describing the mean $\mu$ and variance $\sigma$ that we need to compute from the input data and then $\hat{x}$ which is the normalized input data. The output is given by $y$ which is the combination of the normalized data with the learnable parameters. 

The right column contains the mathematical formulations and can be summarized as follows:
1. For the given input x, we calculate the mean $\mu$ across all input samples.
2. Based on the mean $\mu$, we compute the variances $\sigma$ of each value in the sample.
3. We then normalize the input data based on the computed mean and variance.
4. Finally, we combine the normalized data with the learnable parameters $\gamma$ and $\beta$.

Please remember that Batch Normalization behaves differently at training and test time. In the figure above, we see the behaviour at training time. 

## (Optional) Mount folder in Colab

Uncomment thefollowing cell to mount your gdrive if you are using the notebook in google colab:

In [ ]:
# Use the following lines if you want to use Google Colab
# Don't forget to change to GPU. Runtime --> Change runtime type --> GPU
# NOTE: terminate all other colab sessions that use GPU!
# NOTE 2: Make sure the correct exercise folder (e.g homework_07) is given.
# NOTE 3: For simplicity, create a folder "DL_homeworks" within your main drive folder, and put the exercise there.

"""
import os
import pickle

import os
from google.colab import drive

def mount_gdrive(gdrive_path='/content/gdrive/MyDrive/DL_homeworks/homework_07'):
    # This will mount your google drive under 'MyDrive'
    drive.mount('/content/gdrive', force_remount=True)
    # In order to access the files in this notebook we have to navigate to the correct folder
    os.chdir(gdrive_path)
    # Let's test the setup by simply checking if all paths are present
    present = True
    necessary_paths= ['exercise_code']
    files = os.listdir()
    for path in necessary_paths:
        if path not in files:
            present = False
    if not present:
        print("Can't find all required folders. You either installed the notebook in the wrong path or didn't upload the full exercise folder. You can check your paths in the file overview on the left.")
    else:
        print('Google colab setup complete! Mounted folders:')
        print(files)

# Specify your gdrive path here. If you simply created a 'DL_homeworks' folder in the main directory that is the correct path
mount_gdrive('/content/gdrive/MyDrive/DL_homeworks/homework_07')

# Relevant packages. NOTE: Be patient, it will take a few minutes.
!python -m pip install  tensorboard==2.8.0 pytorch-lightning==1.6.0 torchtext
"""

## 1.2 Batch Normalization: Implementation

In [1]:
# As usual, a bit of setup

import time
import numpy as np
import matplotlib.pyplot as plt
from exercise_code.layers import *
from exercise_code.tests import *
import torch.nn as nn
import torch
import torchvision
import torchvision.transforms as transforms
import pytorch_lightning as pl
import os
import shutil
from pytorch_lightning.loggers import TensorBoardLogger

from exercise_code.networks.BatchNormModel import SimpleNetwork, BatchNormNetwork, DropoutNetwork

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# supress cluttering warnings in solutions
import warnings
warnings.filterwarnings('ignore')

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

### Batch normalization: Forward Pass

<div class="alert alert-success">
    <h3>Task: Check Code</h3>
    <p>In the file <code>exercise_code/layers.py </code>, we have implemented the <code>batchnorm_forward</code> function. Read this implementation and make sure you understand what batch normalization is doing. Then execute the following cells to test the implementation.
 </p>
</div>

In [2]:
# Check the training-time forward pass by checking means and variances
# of features both before and after batch normalization

# Simulate the forward pass for a two-layer network
N, D1, D2, D3 = 200, 50, 60, 3
X = np.random.randn(N, D1)
W1 = np.random.randn(D1, D2)
W2 = np.random.randn(D2, D3)
a = np.maximum(0, X.dot(W1)).dot(W2)

print('Before batch normalization:')
print('  means: ', a.mean(axis=0))
print('  stds: ', a.std(axis=0), '\n')

# Means should be close to zero and stds close to one
print('After batch normalization with (gamma=1, beta=0)')
a_norm, _ = batchnorm_forward(a, np.ones(D3), np.zeros(D3), {'mode': 'train'})
print('  mean: ', a_norm.mean(axis=0))
print('  std: ', a_norm.std(axis=0) , '\n')

# Now means should be close to beta and stds close to gamma
gamma = np.asarray([1.0, 2.0, 3.0])
beta = np.asarray([11.0, 12.0, 13.0])
a_norm, _ = batchnorm_forward(a, gamma, beta, {'mode': 'train'})
print('After batch normalization with (nontrivial gamma, beta)')
print('  means: ', a_norm.mean(axis=0))
print('  stds: ', a_norm.std(axis=0) )

Before batch normalization:
  means:  [ 8.59292588 27.92306639 -8.51749491]
  stds:  [35.68115257 26.10258544 35.24304226] 

After batch normalization with (gamma=1, beta=0)
  mean:  [ 2.39253062e-16 -9.69224700e-16  4.01068068e-17]
  std:  [1.         0.99999999 1.        ] 

After batch normalization with (nontrivial gamma, beta)
  means:  [11. 12. 13.]
  stds:  [1.         1.99999999 2.99999999]


Since the mean and variances in batch norm are computed in training time,
before invoking the test-time forward pass run the training-time
forward pass (previous cell) many times to warm up the running averages. Then
checking the means and variances of activations for a test-time
forward pass.

In [3]:
# Check the test-time forward pass by checking means and variances 
# of features after batch normalization

N, D1, D2, D3 = 200, 50, 60, 3
W1 = np.random.randn(D1, D2)
W2 = np.random.randn(D2, D3)

bn_param = {'mode': 'train'}
gamma = np.ones(D3)
beta = np.zeros(D3)
for t in range(50):
    X = np.random.randn(N, D1)
    a = np.maximum(0, X.dot(W1)).dot(W2)
    batchnorm_forward(a, gamma, beta, bn_param)
bn_param['mode'] = 'test'
X = np.random.randn(N, D1)
a = np.maximum(0, X.dot(W1)).dot(W2)
a_norm, _ = batchnorm_forward(a, gamma, beta, bn_param)

# Means should be close to zero and stds close to one, but will be
# noisier than training-time forward passes.
print('After batch normalization (test-time):')
print('  means: ', a_norm.mean(axis=0))
print('  stds: ', a_norm.std(axis=0))

After batch normalization (test-time):
  means:  [-0.00860989  0.12967537 -0.0052237 ]
  stds:  [1.01008661 0.99859765 0.97590471]


### Batch Normalization: Backward Pass
Since batch normalization is realized by a more complex function of learnable parameters, it is a good exercise to train your backprop skills through this computational graph.

<div class="alert alert-info">
    <h3>Task: Implement</h3>
    <p>Open <code>exercise_code/layer.py</code> and implement the backward pass for Batch Normalization in the function <code> batchnorm_backward() </code>.
    </p>
    <p> To derive the backward pass you should write out the computation graph for batch normalization and backprop through each of the intermediate nodes. Some intermediates may have multiple outgoing branches; make sure to sum gradients across these branches in the backward pass. You can stay close to the forward pass implementation we have provided for you, i.e. go line by line backward.
    </p>
    <p> Once you have finished, run the following to numerically check your backward pass.
    </p>
</div>

In [4]:
# Gradient check batchnorm backward pass

N, D = 4, 5
x = 5 * np.random.randn(N, D) + 12
gamma = np.random.randn(D)
beta = np.random.randn(D)
dout = np.random.randn(N, D)

bn_param = {'mode': 'train'}

fx = lambda x: batchnorm_forward(x, gamma, beta, bn_param)[0]
fg = lambda a: batchnorm_forward(x, gamma, beta, bn_param)[0]
fb = lambda b: batchnorm_forward(x, gamma, beta, bn_param)[0]

dx_num = eval_numerical_gradient_array(fx, x, dout)
da_num = eval_numerical_gradient_array(fg, gamma, dout)
db_num = eval_numerical_gradient_array(fb, beta, dout)

_, cache = batchnorm_forward(x, gamma, beta, bn_param)
dx, dgamma, dbeta = batchnorm_backward(dout, cache)

print('dx error: ', rel_error(dx_num, dx))
print('dgamma error: ', rel_error(da_num, dgamma))
print('dbeta error: ', rel_error(db_num, dbeta))

dx error:  1.0593391199640983e-09
dgamma error:  1.421916445020242e-10
dbeta error:  5.475049990632359e-12


## 1.3 Using Batch Normalization with PyTorch Lightning

Now that we have seen the implementation of Batch Normalization, it is interesting to see how it would affect the overall Model Performance. Since you have already worked with PyTorch Lightning in the last exercise, you have seen how easy it makes our lives. As an experiment, we will use a simple Fully Connected Network in PyTorch Lightning here.

### Setup TensorBoard
In homework_06 you've already learned how to use TensorBoard. Let's use it again to make the debugging of our network and training process more convenient. Throughout this notebook, feel free to add further logs or visualizations to your TensorBoard.

In [5]:
# Few Hyperparameters before we start things off
hidden_dim = 200
batch_size = 50

logdir = './batch_norm_logs'
if os.path.exists(logdir):
    # We delete the logs on the first run
    shutil.rmtree(logdir)
os.mkdir(logdir)

epochs = 5
learning_rate = 0.00005

In [6]:
%load_ext tensorboard
!/Users/tigrangaplanyan/anaconda3/envs/Homeworks/bin/tensorboard --logdir batch_norm_logs

TensorFlow installation not found - running with reduced feature set.
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.9.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C


### Train a model without Batch Normalization. 

<div class="alert alert-success">
    <h3>Task: Check Code</h3>
    <p>Let us first start with a simple network which does not make use of Batch Normalization. We have already implemented the a simple network <code>SimpleNetwork</code> in <code>exercise_code/BatchNormModel.py</code>. Feel free to check it out and play around with the parameters. The cell below is setting up a short trainings process for this network.
 </p>
</div>

In [7]:
# train
model = SimpleNetwork(hidden_dim=hidden_dim, batch_size=batch_size, learning_rate=learning_rate)
# Creating a logging object
simple_network_logger = TensorBoardLogger(
    save_dir=logdir,
    name='simple_network'
)
trainer = pl.Trainer(max_epochs=epochs, logger=simple_network_logger)

trainer.fit(model)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ./batch_norm_logs/simple_network

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 159 K 
-------------------------------------
159 K     Trainable params
0         Non-trainable params
159 K     Total params
0.636     Total estimated model params size (MB)


Epoch 0: 100%|██████████████████████| 960/960 [00:08<00:00, 115.57it/s, v_num=0]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  38%|██████▏         | 92/240 [00:00<00:00, 161.77it/s]


Validation DataLoader 0:  79%|███████████▊   | 189/240 [00:01<00:00, 165.83it/s]


Validation DataLoader 0: 100%|███████████████| 240/240 [00:01<00:00, 167.20it/s]Val-Acc=0.7602499723434448

Epoch 1: 100%|█| 960/960 [00:08<00:00, 116.85it/s, v_num=0, val/loss=0.723, val/
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  18%|██▊             | 42/240 [00:00<00:01, 179.79it/s]


Validation DataLoader 0:  58%|████████▋      | 139/240 [00:00<00:00, 176.88it/s]


...

Validation DataLoader 0: 100%|███████████████| 240/240 [00:01<00:00, 170.05it/s]Val-Acc=3.205749988555908

Epoch 4: 100%|█| 960/960 [00:08<00:00, 117.46it/s, v_num=0, val/loss=0.586, val/
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  36%|█████▋          | 86/240 [00:00<00:00, 164.94it/s]


Validation DataLoader 0:  76%|███████████▍   | 183/240 [00:01<00:00, 165.09it/s]


Validation DataLoader 0: 100%|███████████████| 240/240 [00:01<00:00, 165.04it/s]Val-Acc=4.038166522979736

Epoch 4: 100%|█| 960/960 [00:09<00:00, 98.06it/s, v_num=0, val/loss=0.565, val/a

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|█| 960/960 [00:09<00:00, 97.98it/s, v_num=0, val/loss=0.565, val/a


### Train a model incl. Batch Normalization

<div class="alert alert-success">
    <h3>Task: Check Code</h3>
    <p> Now that we have already seen how our simple network should work, let us look at a model that is actually using Batch Normalization. Again, we provide you with such a model <code>BatchNormNetwork</code> in <code>exercise_code/BatchNormModel.py</code>. Same as before: Feel free to check it out and play around with the parameters. The cell below is setting up a short trainings process for this model. 
 </p>
</div>

In [8]:
model = BatchNormNetwork(hidden_dim=hidden_dim, batch_size=batch_size, learning_rate=learning_rate)
batch_norm_network_logger = TensorBoardLogger(
    save_dir=logdir,
    name='batch_norm_network'
)
trainer = pl.Trainer(max_epochs=epochs, logger=batch_norm_network_logger)
trainer.fit(model)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ./batch_norm_logs/batch_norm_network

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 159 K 
-------------------------------------
159 K     Trainable params
0         Non-trainable params
159 K     Total params
0.638     Total estimated model params size (MB)


Epoch 0: 100%|██████████████████████| 960/960 [00:09<00:00, 104.13it/s, v_num=0]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  38%|██████▏         | 92/240 [00:00<00:00, 153.34it/s]


Validation DataLoader 0:  79%|███████████▊   | 189/240 [00:01<00:00, 158.00it/s]


Validation DataLoader 0: 100%|███████████████| 240/240 [00:01<00:00, 158.57it/s]Val-Acc=4.864833354949951

Epoch 1: 100%|█| 960/960 [00:09<00:00, 99.90it/s, v_num=0, val/loss=0.563, val/a
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  18%|██▊             | 42/240 [00:00<00:01, 161.65it/s]


Validation DataLoader 0:  58%|████████▋      | 139/240 [00:00<00:00, 160.80it/s]


...

Validation DataLoader 0: 100%|███████████████| 240/240 [00:01<00:00, 166.22it/s]Val-Acc=7.42425012588501

Epoch 4: 100%|█| 960/960 [00:08<00:00, 107.46it/s, v_num=0, val/loss=0.516, val/
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  36%|█████▋          | 86/240 [00:00<00:00, 158.67it/s]


Validation DataLoader 0:  76%|███████████▍   | 183/240 [00:01<00:00, 156.52it/s]


Validation DataLoader 0: 100%|███████████████| 240/240 [00:01<00:00, 157.98it/s]Val-Acc=8.289833068847656

Epoch 4: 100%|█| 960/960 [00:10<00:00, 88.35it/s, v_num=0, val/loss=0.502, val/a

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|█| 960/960 [00:10<00:00, 88.27it/s, v_num=0, val/loss=0.502, val/a


### Observations
Take a look at TensorBoard to compare the performance of both networks:

In [9]:
%load_ext tensorboard
!/Users/tigrangaplanyan/anaconda3/envs/Homeworks/bin/tensorboard --logdir batch_norm_logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
TensorFlow installation not found - running with reduced feature set.
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.9.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C


As you can see, using Batch Normalization resulted in better performance. You can easily observe lower validation loss and higher validation accuracy from the graphs. Batch Norm in general is helpful since it would lead to faster model training.

Batch Normalization has other related benefits, for instance, it provides a bit of regularization. However, we also look for other methods of regularization such as Dropout. So in the second part of this notebook, let's have a more detailed look at the effect of Dropout.

# 2. Dropout

## 2.1 What is Dropout

Dropout [1] is a technique for regularizing neural networks by randomly setting some features to zero during the forward pass. While training, traditional dropout is implemented by only keeping a neuron active with some probability p
(a hyperparameter), or setting it to zero otherwise. However, a modification of dropout called "inverted dropout" drops neurons with a given probability. You can read about the difference [here](https://machinelearning.wtf/terms/inverted-dropout/). The Dropout technique would help your Neural Network to perform better on Test data.

We want to repeat the approach that we saw above for Batch Normalization, but this time for Dropout. Let us thus first have a look at the implementation and then compare two networks with each other where one is using Dropout and one is not. 

[1] Srivastava et al, "Dropout: A Simple Way to Prevent Neural Networks from Overfitting", 2014

![](https://drive.google.com/uc?export=view&id=1dQEuk9DwZm3Pe8nBNrJo9FpV17jUMew5)

## 2.2 Inverted Dropout Implementation

### Inverted Dropout: Forward Pass

The dropout method is a little less complex to implement than the Batch Normalization, hence we ask you to implement both, the forward and the backward pass. Let us start with the forward pass:

<div class="alert alert-info">
    <h3>Task: Implement</h3>
    <p> In the file <code>exercise_code/layers.py</code>, implement the forward pass for Dropout in <code>dropout_forward()</code>. Since Dropout behaves differently during training and testing, make sure to implement the operation for both modes.
    </p>
    <p> Once you have done so, run the cell below to test your implementation.
    </p>
</div>

In [10]:
x = np.random.randn(500, 500) + 10
# Let us use different dropout values(p) for our dropout layer and see their effects
for p in [0.3, 0.6, 0.75]:
    out, _ = dropout_forward(x, {'mode': 'train', 'p': p})
    out_test, _ = dropout_forward(x, {'mode': 'test', 'p': p})

    print('Running tests with p = ', p)
    print('Mean of input: ', x.mean())
    print('Mean of train-time output: ', out.mean())
    print('Mean of test-time output: ', out_test.mean())
    print('Fraction of train-time output set to zero: ', (out == 0).mean())
    print('Fraction of test-time output set to zero: ', (out_test == 0).mean())
    print()

Running tests with p =  0.3
Mean of input:  10.003729902741311
Mean of train-time output:  10.011381833687244
Mean of test-time output:  10.003729902741311
Fraction of train-time output set to zero:  0.699812
Fraction of test-time output set to zero:  0.0

Running tests with p =  0.6
Mean of input:  10.003729902741311
Mean of train-time output:  9.992309269734971
Mean of test-time output:  10.003729902741311
Fraction of train-time output set to zero:  0.4007
Fraction of test-time output set to zero:  0.0

Running tests with p =  0.75
Mean of input:  10.003729902741311
Mean of train-time output:  9.996677979616209
Mean of test-time output:  10.003729902741311
Fraction of train-time output set to zero:  0.250468
Fraction of test-time output set to zero:  0.0



### Inverted Dropout: Backward Pass

<div class="alert alert-info">
    <h3>Task: Implement</h3>
    <p> In the file <code>exercise_code/layers.py</code>, implement the backward pass for dropout in <code>dropout_backward()</code>. After doing so, run the following cell to numerically gradient-check your implementation.
    </p>
</div>

In [11]:
x = np.random.randn(10, 10) + 10
dout = np.random.randn(*x.shape)

dropout_param = {'mode': 'train', 'p': 0.8, 'seed': 123}
out, cache = dropout_forward(x, dropout_param)
dx = dropout_backward(dout, cache)
dx_num = eval_numerical_gradient_array(lambda xx: dropout_forward(xx, dropout_param)[0], x, dout)

print('dx relative error: ', rel_error(dx, dx_num))

dx relative error:  5.445608632586474e-11


## 2.3 Using Dropout with PyTorch Lightning

Same experiment as for Batch Normalization: We will train a pair of two-layer networks on a training dataset where one network will use no Dropout and one will use a Dropout probability of 0.75. We will then visualize the training and validation accuracies of the two networks over time.

### Setup TensorBoard

In homework_06 you've already learned how to use TensorBoard. Let's use it again to make the debugging of our network and training process more convenient. Throughout this notebook, feel free to add further logs or visualizations to your TensorBoard.

In [12]:
# Few Hyperparameters before we start things off
hidden_dim = 200
batch_size = 50

epochs = 5
learning_rate = 0.00005

logdir = './dropout_logs'
if os.path.exists(logdir):
    # We delete the logs on the first run
    shutil.rmtree(logdir)
os.mkdir(logdir)

In [13]:
%load_ext tensorboard
!/Users/tigrangaplanyan/anaconda3/envs/Homeworks/bin/tensorboard --logdir dropout_logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
TensorFlow installation not found - running with reduced feature set.
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.9.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C


<div class="alert alert-success">
    <h3>Task: Check Code</h3>
    <p> As before, we have already implemented those two networks for you. You may check them out in <code>exercise_code/BatchNormModel.py</code>. As always, feel free to play around with the parameters here. Run the following two cells to setup both models and train them for a few epochs in order to compare the performance with and without Dropout. 
 </p>
</div>

### Train a model without Dropout

Let us first start with a simple network `SimpleNetwork` which does not make use of Dropout.

In [14]:
# train a model without Dropout
model = SimpleNetwork(hidden_dim=hidden_dim, batch_size=batch_size, learning_rate=learning_rate)
simple_network_logger = TensorBoardLogger(
    save_dir=logdir,
    name='simple_network'
)
trainer = pl.Trainer(max_epochs=epochs, logger=simple_network_logger)

trainer.fit(model)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ./dropout_logs/simple_network

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 159 K 
-------------------------------------
159 K     Trainable params
0         Non-trainable params
159 K     Total params
0.636     Total estimated model params size (MB)


Epoch 0: 100%|██████████████████████| 960/960 [00:07<00:00, 120.41it/s, v_num=0]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  38%|██████▏         | 92/240 [00:00<00:00, 159.52it/s]


Validation DataLoader 0:  79%|███████████▊   | 189/240 [00:01<00:00, 162.22it/s]


Validation DataLoader 0: 100%|███████████████| 240/240 [00:01<00:00, 163.25it/s]Val-Acc=9.04675006866455

Epoch 1: 100%|█| 960/960 [00:08<00:00, 116.73it/s, v_num=0, val/loss=0.523, val/
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  18%|██▊             | 42/240 [00:00<00:01, 164.95it/s]


Validation DataLoader 0:  58%|████████▋      | 139/240 [00:00<00:00, 165.24it/s]


...

Validation DataLoader 0: 100%|███████████████| 240/240 [00:01<00:00, 174.02it/s]Val-Acc=11.485750198364258

Epoch 4: 100%|█| 960/960 [00:08<00:00, 119.50it/s, v_num=0, val/loss=0.527, val/
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  36%|█████▋          | 86/240 [00:00<00:00, 171.27it/s]


Validation DataLoader 0:  76%|███████████▍   | 183/240 [00:01<00:00, 172.17it/s]


Validation DataLoader 0: 100%|███████████████| 240/240 [00:01<00:00, 172.75it/s]Val-Acc=12.316499710083008

Epoch 4: 100%|█| 960/960 [00:10<00:00, 94.32it/s, v_num=0, val/loss=0.524, val/a

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|█| 960/960 [00:10<00:00, 94.26it/s, v_num=0, val/loss=0.524, val/a


### Train a model incl. Dropout

Now that we have already seen how our simple network should work, let us look at the model `DropoutNetwork` that is actually using Dropout.

In [15]:
# train a model with Dropout
model = DropoutNetwork(hidden_dim=hidden_dim, batch_size=batch_size, learning_rate=learning_rate,dropout_p=0.75)
dropout_network_logger = TensorBoardLogger(
    save_dir=logdir,
    name='dropout_network'
)
trainer = pl.Trainer(max_epochs=epochs, logger=dropout_network_logger)

trainer.fit(model)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ./dropout_logs/dropout_network

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 159 K 
-------------------------------------
159 K     Trainable params
0         Non-trainable params
159 K     Total params
0.638     Total estimated model params size (MB)


Epoch 0: 100%|███████████████████████| 960/960 [00:09<00:00, 96.55it/s, v_num=0]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  38%|██████▏         | 92/240 [00:00<00:00, 157.05it/s]


Validation DataLoader 0:  79%|███████████▊   | 189/240 [00:01<00:00, 160.80it/s]


Validation DataLoader 0: 100%|███████████████| 240/240 [00:01<00:00, 160.56it/s]Val-Acc=13.109416961669922

Epoch 1: 100%|█| 960/960 [00:09<00:00, 98.10it/s, v_num=0, val/loss=0.534, val/a
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  18%|██▊             | 42/240 [00:00<00:01, 156.07it/s]


Validation DataLoader 0:  58%|████████▋      | 139/240 [00:00<00:00, 155.98it/s]


...

Validation DataLoader 0: 100%|███████████████| 240/240 [00:01<00:00, 166.15it/s]Val-Acc=15.59041690826416

Epoch 4: 100%|█| 960/960 [00:09<00:00, 99.26it/s, v_num=0, val/loss=0.529, val/a
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  36%|█████▋          | 86/240 [00:00<00:00, 166.60it/s]


Validation DataLoader 0:  76%|███████████▍   | 183/240 [00:01<00:00, 166.40it/s]


Validation DataLoader 0: 100%|███████████████| 240/240 [00:01<00:00, 166.36it/s]Val-Acc=16.430665969848633

Epoch 4: 100%|█| 960/960 [00:12<00:00, 77.63it/s, v_num=0, val/loss=0.525, val/a

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|█| 960/960 [00:12<00:00, 77.57it/s, v_num=0, val/loss=0.525, val/a


### Observations

Take a look at TensorBoard to compare the performance of both networks:

In [16]:
%load_ext tensorboard
!/Users/tigrangaplanyan/anaconda3/envs/Homeworks/bin/tensorboard --logdir dropout_logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
TensorFlow installation not found - running with reduced feature set.
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.9.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C


As you can see, by using Dropout we can see that the Training Loss would increase but the model would perform better on the Validation Set. Like Batch Normalization, Dropout also has different behavior at Train and Test time. 